In [11]:
import numpy as np
import pandas as ps
from matplotlib import pyplot as plt

<module 'numpy' from 'D:\\plant image classification\\plant-classsification\\venv\\lib\\site-packages\\numpy\\__init__.py'>
